In [1]:
!pip install onnx onnxruntime num2words

In [2]:
! git clone https://github.com/huggingface/lerobot.git
! cd lerobot && pip install .

fatal: destination path 'lerobot' already exists and is not an empty directory.
Processing /content/lerobot
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for lerobot: filename=lerobot-0.3.4-py3-none-any.whl size=605051 sha256=be50846e2c00df1698349ded5643fceee05df033d7115a5974c0ff8dbdfa5b32
  Stored in directory: /tmp/pip-ephem-wheel-cache-m40dzc5q/wheels/09/b4/fe/75732b1d640db96ba1f856f2b7328b232a03b696a39cb59686
Successfully built lerobot
  Attempting uninstall: lerobot
    Found existing installation: lerobot 0.3.4
    Uninstalling lerobot-0.3.4:
      Successfully uninstalled lerobot-0.3.4


In [3]:
%pip install -q ipdb
%env PYTHONBREAKPOINT=IPython.core.debugger.set_trace  # makes breakpoint() use ipdb
%pdb on

env: PYTHONBREAKPOINT=IPython.core.debugger.set_trace  # makes breakpoint() use ipdb
Automatic pdb calling has been turned ON


In [4]:
from lerobot.policies.smolvla.modeling_smolvla import SmolVLAPolicy
from lerobot.policies.smolvla.configuration_smolvla import SmolVLAConfig

config = SmolVLAConfig
policy = SmolVLAPolicy.from_pretrained("lerobot/smolvla_base")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Loading  HuggingFaceTB/SmolVLM2-500M-Video-Instruct weights ...


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Reducing the number of VLM layers to 16 ...
[standardise_state_dict] 'normalize_inputs.buffer_observation_state.mean'  ←  ['normalize_inputs.so100-red_buffer_observation_state.mean', 'normalize_inputs.so100_buffer_observation_state.mean']
[standardise_state_dict] 'normalize_inputs.buffer_observation_state.std'  ←  ['normalize_inputs.so100-red_buffer_observation_state.std', 'normalize_inputs.so100_buffer_observation_state.std']
[standardise_state_dict] 'normalize_targets.buffer_action.mean'  ←  ['normalize_targets.so100-red_buffer_action.mean', 'normalize_targets.so100_buffer_action.mean']
[standardise_state_dict] 'normalize_targets.buffer_action.std'  ←  ['normalize_targets.so100-red_buffer_action.std', 'normalize_targets.so100_buffer_action.std']
[standardise_state_dict] 'unnormalize_outputs.buffer_action.mean'  ←  ['unnormalize_outputs.so100-red_buffer_action.mean', 'unnormalize_outputs.so100_buffer_action.mean']
[standardise_state_dict] 'unnormalize_outputs.buffer_action.std'  ←  ['

In [5]:
import torch
from torch import nn
from lerobot.policies.smolvla.smolvlm_with_expert import SmolVLMWithExpertModel

from typing import Optional, List

class SmolVLMForONNXExport(nn.Module):
    def __init__(self, model_id, **kwargs):
        super().__init__()
        self.model = policy.model.vlm_with_expert

        self.config = SmolVLAConfig

        # Bake in the configuration for the static graph
        self.attention_mode = self.config.vlm_model_name
        self.num_vlm_layers = self.model.num_vlm_layers
        self.num_expert_layers = self.model.num_expert_layers
        self.fill_kv_cache = False
        self.use_cache = True

    def forward(
          self,
          vlm_embeds: torch.Tensor,
          expert_embeds: torch.Tensor,
          attention_mask: torch.Tensor,
          position_ids: torch.LongTensor,
          # Flattened past_key_values: one tensor for keys, one for values per layer
          *past_key_values_flat: torch.Tensor):


          inputs_embeds = [vlm_embeds, expert_embeds]

          past_key_values = {}
          if past_key_values_flat:
              # Logic to reconstruct the dictionary from the flat tuple of tensors
              # This assumes a consistent ordering: (key_layer0, val_layer0, key_layer1, val_layer1, ...)
              for i in range(0, len(past_key_values_flat), 2):
                  layer_idx = i // 2
                  past_key_values[layer_idx] = {
                      "key_states": past_key_values_flat[i],
                      "value_states": past_key_values_flat[i+1],
                  }
          else:
              past_key_values = None


          final_embeds_list, new_past_key_values = self.model.forward(
              attention_mask=attention_mask,
              position_ids=position_ids,
              past_key_values=past_key_values,
              inputs_embeds=inputs_embeds,
              use_cache=self.use_cache,
              fill_kv_cache=self.fill_kv_cache,
          )

          # --- 3. Flatten Outputs ---
          # The output must be a flat tuple of tensors
          vlm_output_embeds = final_embeds_list[0]
          expert_output_embeds = final_embeds_list[1] # expects to be None

          present_key_values_flat = []
          if new_past_key_values:
            for i in range(self.num_vlm_layers):
                if i in new_past_key_values:
                    present_key_values_flat.append(new_past_key_values[i]["key_states"])
                    present_key_values_flat.append(new_past_key_values[i]["value_states"])

          return (vlm_output_embeds, expert_output_embeds, *present_key_values_flat)

In [6]:
import torch

onnx_model = SmolVLMForONNXExport("")

In [7]:
onnx_model.eval()

onnx_model.fill_kv_cache = True

onnx_model.to("cuda:0")

# output_names = ["vlm_output_embeds", "expert_output_embeds"]
output_names = ["vlm_output_embeds"]
num_layers = onnx_model.model.num_vlm_layers
for i in range(num_layers):
    output_names.append(f"present_key_{i}")
    output_names.append(f"present_value_{i}")

vlm_hidden_dim = onnx_model.model.config.text_config.hidden_size
expert_hidden_dim = onnx_model.model.expert_hidden_size

batch_size = 1
vlm_seq_len = 256
expert_seq_len = 16
total_seq_len = vlm_seq_len + expert_seq_len

dummy_vlm_embeds = torch.randn(batch_size, vlm_seq_len, 960, device="cuda:0", dtype=torch.bfloat16)
# dummy_expert_embeds = torch.randn(batch_size, expert_seq_len, 320, device="cuda:0", dtype=torch.bfloat16) # input_embeds
dummy_expert_embeds = torch.randn(batch_size, expert_seq_len, expert_hidden_dim, device="cuda:0", dtype=torch.bfloat16) # input_embeds
# dummy_expert_embeds = torch.randn(batch_size, 320, expert_hidden_dim, device="cuda:0", dtype=torch.bfloat16) # input_embeds
dummy_attn_mask = torch.ones(batch_size, total_seq_len, total_seq_len, device="cuda:0", dtype=torch.bool)
dummy_pos_ids = torch.arange(total_seq_len, device="cuda:0").unsqueeze(0)


# dummy_inputs = (dummy_vlm_embeds, dummy_expert_embeds, dummy_attn_mask, dummy_pos_ids)
dummy_inputs = (dummy_vlm_embeds, None, dummy_attn_mask, dummy_pos_ids)

# we ant past_key_values from it, the second output



In [8]:
torch.onnx.export(
    onnx_model,
    dummy_inputs,
    "smolvlm_expert_prefill.onnx",
    input_names=["vlm_embeds", "expert_embeds", "attention_mask", "position_ids"],
    output_names=output_names,
    opset_version=17
)

/usr/local/lib/python3.12/dist-packages/lerobot/policies/smolvla/smolvlm_with_expert.py:237: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if seq_len < position_ids.shape[1]:


In [9]:
_, _, *past_key_values = onnx_model.forward(*dummy_inputs)

In [10]:
past_key_values[0].shape

torch.Size([1, 256, 5, 64])

In [11]:
import torch


onnx_model = SmolVLMForONNXExport("")

In [12]:

onnx_model.to("cuda:0")
onnx_model.eval()
onnx_model.fill_kv_cache = False
onnx_model.use_cache = True
current_seq_len = 1
past_seq_len = 256

batch_size = 1

vlm_hidden_dim = onnx_model.model.config.text_config.hidden_size
expert_hidden_dim = onnx_model.model.expert_hidden_size
num_layers = onnx_model.model.num_vlm_layers
num_kv_heads = onnx_model.model.config.text_config.num_key_value_heads
head_dim = vlm_hidden_dim // onnx_model.model.config.text_config.num_attention_heads

dummy_vlm_embeds = torch.randn(batch_size, current_seq_len, 960, device="cuda:0", dtype=torch.bfloat16)
dummy_expert_embeds = torch.randn(batch_size, current_seq_len, expert_hidden_dim, device="cuda:0", dtype=torch.bfloat16)
# dummy_attn_mask = torch.ones(batch_size, current_seq_len, past_seq_len + current_seq_len, device="cuda:0", dtype=torch.bool)
dummy_attn_mask = torch.ones(batch_size, current_seq_len, current_seq_len, device="cuda:0", dtype=torch.bool)
# dummy_attn_mask = torch.ones(batch_size, 1, past_seq_len, past_seq_len + current_seq_len + 1, device="cuda:0", dtype=torch.bool)
# dummy_attn_mask = torch.ones(batch_size, 1, past_seq_len + current_seq_len, device="cuda:0", dtype=torch.bool)
# dummy_pos_ids = torch.arange(past_seq_len + current_seq_len, device="cuda:0").unsqueeze(0)
dummy_pos_ids = torch.arange(current_seq_len, device="cuda:0").unsqueeze(0)

print(past_seq_len + current_seq_len)
print(total_seq_len)

input_names = ["vlm_embeds", "expert_embeds", "attention_mask", "position_ids"]

# past_key_values_flat = []
# for _ in range(num_layers):
#     key_shape = (batch_size, num_kv_heads, past_seq_len, head_dim)
#     val_shape = (batch_size, num_kv_heads, past_seq_len, head_dim)
#     past_key_values_flat.append(torch.randn(key_shape, device="cuda:0", dtype=torch.bfloat16))
#     past_key_values_flat.append(torch.randn(val_shape, device="cuda:0", dtype=torch.bfloat16))

dummy_inputs = (
    # dummy_vlm_embeds,
    None,
    dummy_expert_embeds,
    dummy_attn_mask,
    dummy_pos_ids,
    *past_key_values
    # None
)


for i in range(num_layers):
    input_names.append(f"past_key_{i}")
    input_names.append(f"past_value_{i}")


# Outputs include the embeddings and the *updated* KV cache
# output_names = ["vlm_output_embeds", "expert_output_embeds"]
output_names = ["expert_output_embeds"]
for i in range(num_layers):
    # The output names must be consistent for the next iteration
    output_names.append(f"present_key_{i}")
    output_names.append(f"present_value_{i}")

# dynamic_axes = {"vlm_embeds": {}, "expert_embeds": {}} # Add other static inputs if needed
# for i in range(num_layers):
#     # The '2' axis is the sequence length dimension in the KV cache shape
#     # (batch_size, num_heads, sequence_length, head_dim)
#     dynamic_axes[f"past_key_{i}"] = {2: "past_sequence_length"}
#     dynamic_axes[f"past_value_{i}"] = {2: "past_sequence_length"}
#     dynamic_axes[f"present_key_{i}"] = {2: "total_sequence_length"}
#     dynamic_axes[f"present_value_{i}"] = {2: "total_sequence_length"}

# we need output_embeds - first output
torch.onnx.export(
    onnx_model,
    dummy_inputs,
    "smolvlm_expert_decode.onnx",
    input_names=input_names,
    output_names=output_names,
    # dynamic_axes=dynamic_axes,
    opset_version=17,
    do_constant_folding=True,
)


257
272


In [13]:
import os, onnx

out_name = None
src_name = "smolvlm_expert_prefill.onnx"
out_dir = "splited"
os.makedirs(out_dir, exist_ok=True)
base = out_name or (os.path.splitext(os.path.basename(src_name))[0] + "_ext.onnx")
out_path = os.path.join(out_dir, base)

model = onnx.load(src_name)
onnx.save_model(
        model,
        out_path,
        save_as_external_data=True,
        all_tensors_to_one_file=True,
        location="weights.bin",
        size_threshold=0,
        convert_attribute=True
    )

In [14]:
import os, onnx

out_name = None
src_name = "smolvlm_expert_decode.onnx"
out_dir = "splited_decoder"
os.makedirs(out_dir, exist_ok=True)
base = out_name or (os.path.splitext(os.path.basename(src_name))[0] + "_ext.onnx")
out_path = os.path.join(out_dir, base)

model = onnx.load(src_name)
onnx.save_model(
        model,
        out_path,
        save_as_external_data=True,
        all_tensors_to_one_file=True,
        location="weights.bin",
        size_threshold=0,
        convert_attribute=True
    )